# Next Step


Discuss general algorithm: need to approximate gradient for back propagation. Then present gradient approximation methods.
- Closed-Form Decisions
- Linear Decision Objective
- Quadratic Decision Objective
- Generic Decision Objective

Gradient Free Methods

Experiments

Methods to compare:
- Two-stage: prediction then decision, prediction then fair decision, fair prediction then decision, fair prediction then fair decision
- DFL: DFL version of each of the above two-stage settings


Performance measures to report:
- Prediction accuracy: mean square errors of $r$ and $\hat{r}$
- Decision accuracy: mean square errors of $d(r)$ and $d(\hat{r})$
- Prediction fairness: prediction fairness measure of $\hat{r}$
- Decision fairness: decision fairness measure of $d(\hat(r))$
- Runtime of algorithm

In [25]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
import matplotlib.pyplot as plt
import pandas as pd
import sys
sys.path.insert(0, 'E:\\User\\Stevens\\Code\\The Paper\\algorithm')
from torch.utils.data import Dataset, DataLoader


import warnings
warnings.filterwarnings("ignore")

from myutil import *
from features import *

In [26]:
df = pd.read_csv('data/data.csv')

columns_to_keep = [
    'risk_score_t', 'program_enrolled_t', 'cost_t', 'cost_avoidable_t', 'race', 'dem_female', 'gagne_sum_tm1', 'gagne_sum_t', 
    'risk_score_percentile', 'screening_eligible', 'avoidable_cost_mapped', 'propensity_score', 'g_binary', 
    'g_continuous', 'utility_binary', 'utility_continuous'
]
# for race 0 is white, 1 is black
df_stat = df[columns_to_keep]
df_feature = df[[col for col in df.columns if col not in columns_to_keep]]

# Replace all values less than 0.1 with 0.1
df['risk_score_t'] = df['risk_score_t'].apply(lambda x: 0.1 if x < 0.1 else x)
df['g_continuous'] = df['g_continuous'].apply(lambda x: 0.1 if x < 0.1 else x)

# subset a sample of 5000 rows of df
# df = df.sample(n=10000, random_state=1)

df.shape

(48784, 168)

In [27]:
# Define input variables for DFL
feats = df[get_all_features(df)].values
risk = df['risk_score_t'].values
gainF = df['g_continuous'].values
pScore = df['propensity_score'].values
cost = np.ones(risk.shape)

alpha = 0.5
Q = 100

utility = risk * gainF * pScore

risk.mean()

4.394240983699624

# Prediction Stage

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
feats = scaler.fit_transform(feats)

# Train the Prediction Model
class RiskDataset(Dataset):
    def __init__(self, features, risks):
        self.features = torch.FloatTensor(features)
        self.risks = torch.FloatTensor(risks).reshape(-1, 1)
        
    def __len__(self):
        return len(self.features)
        
    def __getitem__(self, idx):
        return self.features[idx], self.risks[idx]
    
class RiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1),

            nn.Softplus()
        )
    
    def forward(self, x):
        return self.model(x)

# Training function
def train_model(features, risks, epochs=10, batch_size=32):
    dataset = RiskDataset(features, risks)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    model = RiskPredictor(features.shape[1])
    model.train()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    
    for epoch in range(epochs):
        for batch_features, batch_risks in dataloader:
            optimizer.zero_grad()
            predictions = model(batch_features)
            loss = criterion(predictions, batch_risks)
            loss.backward()
            optimizer.step()
            
        if (epoch + 1) % 5 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
    
    return model



# model = train_model(feats, risk)
# torch.save(model.state_dict(), 'risk_predictor_model.pth')

# Load the model from local
model = RiskPredictor(feats.shape[1])
model.load_state_dict(torch.load('risk_predictor_model.pth'))
model.eval()

pred_risk = model(torch.FloatTensor(feats)).detach().numpy().flatten()

pred_risk.mean(), risk.mean()

(2.7172887, 4.394240983699624)

# Solve Optimization Problem

In [29]:
def AlphaFairness(util,alpha):
    if alpha == 1:
        return np.sum(np.log(util))
    elif alpha == 0:
        return np.sum(util)
    elif alpha == 'inf':
        return np.min(util)
    else:
        return np.sum(util**(1-alpha)/(1-alpha))
    
def solve_optimization(gainF, risk, cost, alpha, Q):
    # Flatten input arrays
    gainF, risk, cost = gainF.flatten(), risk.flatten(), cost.flatten()
    d = cp.Variable(risk.shape, nonneg=True)
    
    utils = cp.multiply(cp.multiply(gainF, risk), d)
    
    if alpha == 'inf':
        # Maximin formulation
        t = cp.Variable()  # auxiliary variable for minimum utility
        objective = cp.Maximize(t)
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q,
            utils >= t  # t is the minimum utility
        ]
    elif alpha == 1:
        # Nash welfare (alpha = 1)
        objective = cp.Maximize(cp.sum(cp.log(utils)))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    elif alpha == 0:
        # Utilitarian welfare (alpha = 0)
        objective = cp.Maximize(cp.sum(utils))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    else:
        # General alpha-fairness
        objective = cp.Maximize(cp.sum(utils**(1-alpha))/(1-alpha) if alpha != 0 
                              else cp.sum(utils))
        constraints = [
            d >= 0,
            # d <= 1,
            cp.sum(cost * d) <= Q
        ]
    
    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.MOSEK, verbose=True, warm_start=True, mosek_params={'MSK_IPAR_LOG': 1})
    
    if problem.status != 'optimal':
        print(f"Warning: Problem status is {problem.status}")
    
    optimal_decision = d.value
    optimal_value = AlphaFairness(optimal_decision * gainF * risk, alpha)
    
    return optimal_decision, optimal_value

In [30]:
# pred_sol,_ = solve_optimization(gainF, predicted_risk, cost, alpha='inf', Q=Q)
# pred_obj = np.sum((risk * gainF * pred_sol)**(1-alpha)/(1-alpha))
# true_obj = np.sum((optimal_decision * gainF * risk)**(1-alpha)/(1-alpha))

In [35]:
def twoStagePTO(model, feats, gainF, risk, cost, Q, alphas=[0.5]):
    """
    Perform a two-stage optimization analysis with predictions and calculate normalized regrets.

    Args:
        model (nn.Module): A regression neural network for risk prediction.
        feats (np.ndarray): Feature array for predictions.
        gainF (np.ndarray): Gain factors.
        risk (np.ndarray): True risk values.
        cost (np.ndarray): Cost constraints.
        Q (float): Budget constraint.
        alphas (list): List of alpha values for fairness.

    Returns:
        pd.DataFrame: A table of prediction risk means, true risk mean, objectives, and normalized regrets.
    """
    from sklearn.preprocessing import StandardScaler
    import pandas as pd

    # Feature scaling
    scaler = StandardScaler()
    feats_scaled = scaler.fit_transform(feats)

    # Predict risks
    model.eval()
    pred_risk = model(torch.FloatTensor(feats_scaled)).detach().numpy().flatten()

    # Initialize result storage
    results = []

    # Iterate over alphas
    for alpha in alphas:
        # Solve optimization problems
        true_sol, _ = solve_optimization(gainF, risk, cost, alpha, Q)
        pred_sol, _ = solve_optimization(gainF, pred_risk, cost, alpha, Q)

        # Calculate true and predicted utilities
        true_utility = risk * gainF * true_sol
        pred_utility = pred_risk * gainF * pred_sol

        # Calculate objectives
        true_obj = AlphaFairness(true_utility, alpha)
        pred_obj = AlphaFairness(pred_utility, alpha)

        # Calculate regret and normalized regret
        regret = true_obj - pred_obj
        normalized_regret = regret / (abs(true_obj) + 1e-7)

        # Collect results for this alpha
        results.append({
            'Alpha': alpha,
            'Predicted Risk Mean': pred_risk.mean(),
            'True Risk Mean': risk.mean(),
            'True Objective': true_obj,
            'Predicted Objective': pred_obj,
            'Normalized Regret': normalized_regret
        })

    # Create a DataFrame for results
    results_df = pd.DataFrame(results)
    print(results_df)
    return results_df


In [41]:
data_sample = df.sample(n=20, random_state=42)
feats_sample = data_sample[get_all_features(data_sample)].values
risk_sample = data_sample['risk_score_t'].values
gainF_sample = data_sample['g_continuous'].values
decision_sample = data_sample['propensity_score'].values
cost_sample = np.ones(risk_sample.shape)

twoStagePTO(model, feats_sample, gainF_sample, risk_sample, cost_sample, Q, alphas=[0,.5,.99,1,2,'inf'])

                                     CVXPY                                     
                                     v1.4.3                                    
(CVXPY) Jan 08 12:19:03 PM: Your problem has 20 variables, 2 constraints, and 0 parameters.
(CVXPY) Jan 08 12:19:03 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jan 08 12:19:03 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jan 08 12:19:03 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jan 08 12:19:03 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jan 08 12:19:03 PM: Compiling problem (target solver=MOSEK).
(CV

,Alpha,Predicted Risk Mean,True Risk Mean,True Objective,Predicted Objective,Normalized Regret
0,0,2.601545,4.537116,19649.519211,1623.189928,0.917393
1,0.5,2.601545,4.537116,417.813131,168.296648,0.597196
2,0.99,2.601545,4.537116,2080.077958,2041.579111,0.018508
3,1,2.601545,4.537116,78.405349,40.911323,0.478208
4,2,2.601545,4.537116,-0.496096,-5.017219,9.113410
5,inf,2.601545,4.537116,34.674789,2.140391,0.938272


regrets

# SPO